In [1]:
import os
os.environ["Spark_home"] = "D:/spark-3.4.1-bin-hadoop3"


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col

In [3]:
spark = SparkSession.builder.appName("movies_Recommendation").master("local").getOrCreate()


In [4]:
df = spark.read.csv("C:/Users/IDEAL/Desktop/ratings.csv", inferSchema=True, header=True)

In [5]:
#df.show()

In [6]:
df.describe().show()

+-------+-----------------+------------------+------------------+--------------------+
|summary|           userId|           movieId|            rating|           timestamp|
+-------+-----------------+------------------+------------------+--------------------+
|  count|         26024289|          26024289|          26024289|            26024289|
|   mean| 135037.090248114|15849.109677040553|3.5280903543608817|1.1712584326913223E9|
| stddev|78176.19722170963|31085.257531391508|1.0654427636662405|2.0528887028185263E8|
|    min|                1|                 1|               0.5|           789652004|
|    max|           270896|            176275|               5.0|          1501829870|
+-------+-----------------+------------------+------------------+--------------------+



In [7]:
(train, test) = df.randomSplit([0.7, 0.3], seed=42)

In [8]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")

In [9]:
model = als.fit(train)

In [11]:
pred = model.transform(test)

In [12]:
pred.show()

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|     1|    858|   5.0|1425941523|  4.317981|
|     1|  58559|   4.0|1425942007| 4.0110464|
|     1|  81834|   5.0|1425942133| 4.4471855|
|     1|  98809|   0.5|1425942640| 3.8321376|
|     2|    339|   5.0| 867041296| 3.6341438|
|     1|   4226|   4.0|1425942228|  4.746791|
|     1|  96821|   5.0|1425941382| 5.3166966|
|     1|  91542|   5.0|1425942618| 3.8924344|
|     1|  59315|   5.0|1425941502| 3.5215898|
|     1|   2959|   4.0|1425941601|  4.369206|
|     2|    648|   4.0| 867039167| 2.7252045|
|     2|   1233|   4.0| 867039820|   3.63178|
|     2|     25|   3.0| 867039168| 2.1871753|
|     2|     32|   2.0| 867039166| 3.5376983|
|     2|    786|   1.0| 867039249|  3.219135|
|     2|     58|   3.0| 867039325|  4.158926|
|     2|    260|   4.0| 867039249|  4.043883|
|     1|   2762|   4.5|1425941300| 3.7820456|
|     2|     79|   4.0| 867039325|

In [14]:
eval = RegressionEvaluator(metricName="rmse",labelCol="rating", predictionCol="prediction")

In [17]:
rmse = eval.evaluate(pred)
print(f"RMSE: {rmse}")

RMSE: nan


In [18]:
#RMSE describe our error in terms of the starts rating column

In [19]:
user_1 = test.filter(test['userId'] == 1).select(['movieId', "userId"])

In [20]:
user_1.show()

+-------+------+
|movieId|userId|
+-------+------+
|    858|     1|
|   2762|     1|
|   2959|     1|
|   4226|     1|
|  54503|     1|
|  58559|     1|
|  59315|     1|
|  81834|     1|
|  91542|     1|
|  96821|     1|
|  98809|     1|
+-------+------+



In [21]:
rec = model.transform(user_1)

In [22]:
rec.orderBy("prediction", ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|  96821|     1| 5.3166966|
|   4226|     1|  4.746791|
|  81834|     1| 4.4471855|
|   2959|     1|  4.369206|
|    858|     1|  4.317981|
|  54503|     1| 4.2595515|
|  58559|     1| 4.0110464|
|  91542|     1| 3.8924344|
|  98809|     1| 3.8321376|
|   2762|     1| 3.7820456|
|  59315|     1| 3.5215898|
+-------+------+----------+

